In [2]:
import numpy as np
from src.reference_solver import solve_reference_weno
from src.dataset_gen import random_ic_fourier
from src.simulate import simulate_steps
from src.neural_flux import NeuralFlux
import tensorflow as tf
import matplotlib.pyplot as plt

In [3]:
model = tf.keras.models.load_model(
    "models/FirstIterate.keras",
    custom_objects={"NeuralFlux": NeuralFlux},
    compile=False,  # strongly recommended for inference/eval
)

In [8]:
def block_average(u_fine: np.ndarray, Ncoarse: int) -> np.ndarray:
    """
    u_fine shape (Nfine,), periodic grid, assume Nfine % Ncoarse == 0.
    """
    Nfine = u_fine.size
    r = Nfine // Ncoarse
    assert Nfine % Ncoarse == 0
    return u_fine.reshape(Ncoarse, r).mean(axis=1)

def make_grids(Nfine: int, Ncoarse: int):
    x_f = np.linspace(0.0, 1.0, Nfine, endpoint=False)
    x_c = np.linspace(0.0, 1.0, Ncoarse, endpoint=False)
    return x_f, x_c

def reference_coarse_trajectory(u0_fine, x_fine, mu, T, Ncoarse, cfl_ref=0.4, save_every=10):
    """
    Returns times_ref (np.ndarray), Uc_ref (list of coarse snapshots)
    """
    t_ref, Ufine_snaps = solve_reference_weno(u0_fine, x_fine, T=T, mu=mu, cfl=cfl_ref, save_every=save_every)
    Uc = [block_average(u, Ncoarse) for u in Ufine_snaps]
    return t_ref, Uc

def l2_error(u_hat: np.ndarray, u_ref: np.ndarray) -> float:
    return float(np.sqrt(np.mean((u_hat - u_ref)**2)))

In [9]:
def eval_case(model, mu, Nfine=2048, Ncoarse=200, T=0.2, seed=0, steps=2000):
    """
    Generates a random IC on fine grid, builds coarse truth from reference,
    then compares NN-FV vs Rusanov-FV on the same coarse grid for a fixed number of steps.

    Returns a dict of final-time RMSEs.
    """
    x_f, x_c = make_grids(Nfine, Ncoarse)
    u0_f = random_ic_fourier(x_f, K=6, seed=seed)
    u0_c = block_average(u0_f, Ncoarse)

    # coarse truth from fine reference
    t_ref, Uc_ref = reference_coarse_trajectory(u0_f, x_f, mu=mu, T=T, Ncoarse=Ncoarse, save_every=10)
    u_ref_final = Uc_ref[-1]

    # simulate coarse with rusanov
    t_r, U_r = simulate_steps(u0_c, x_c, mu=mu, nsteps=steps, model=None, save_every=steps)
    u_r_final = U_r[-1]

    # simulate coarse with NN flux
    t_n, U_n = simulate_steps(u0_c, x_c, mu=mu, nsteps=steps, model=model, save_every=steps)
    u_n_final = U_n[-1]

    return {
        "mu": mu,
        "seed": seed,
        "rmse_rusanov": l2_error(u_r_final, u_ref_final),
        "rmse_nn": l2_error(u_n_final, u_ref_final),
        "ratio_nn_over_rusanov": l2_error(u_n_final, u_ref_final) / (l2_error(u_r_final, u_ref_final) + 1e-12),
    }

def generalization_tests(model, train_mus, holdout_mus, seeds=(0,1,2), **kwargs):
    results = []
    for mu in list(train_mus) + list(holdout_mus):
        for s in seeds:
            results.append(eval_case(model, mu=mu, seed=s, **kwargs))
    return results


In [19]:
train_mus   = [1e-2, 3e-2, 1e-1]
holdout_mus = [5e-3, 2e-1]
results = generalization_tests(model, train_mus, holdout_mus, Ncoarse=256, T=0.2, steps=1500)

In [ ]:
print(results)

{'mu': 0.01, 'seed': 0, 'rmse_rusanov': 0.1893154141870137, 'rmse_nn': 0.18883697842414512, 'ratio_nn_over_rusanov': 0.9974728113612903}
{'mu': 0.01, 'seed': 1, 'rmse_rusanov': 0.2676486880104871, 'rmse_nn': 0.2686060244691083, 'ratio_nn_over_rusanov': 1.0035768397175187}
{'mu': 0.01, 'seed': 2, 'rmse_rusanov': 0.279119222258809, 'rmse_nn': 0.27891671102194404, 'ratio_nn_over_rusanov': 0.9992744633055891}
{'mu': 0.03, 'seed': 0, 'rmse_rusanov': 0.05518969051425164, 'rmse_nn': 0.05441449980482664, 'ratio_nn_over_rusanov': 0.9859540667253647}
{'mu': 0.03, 'seed': 1, 'rmse_rusanov': 0.07979964684217358, 'rmse_nn': 0.07825904516414176, 'ratio_nn_over_rusanov': 0.9806941291098756}
{'mu': 0.03, 'seed': 2, 'rmse_rusanov': 0.08623070800876603, 'rmse_nn': 0.08510316608403472, 'ratio_nn_over_rusanov': 0.9869241253869373}
{'mu': 0.1, 'seed': 0, 'rmse_rusanov': 0.04601758573918609, 'rmse_nn': 0.0463190412537882, 'ratio_nn_over_rusanov': 1.0065508763389743}
{'mu': 0.1, 'seed': 1, 'rmse_rusanov': 0.

In [10]:
def long_horizon_stress(model, mu, seed=0, horizons=(2000, 5000, 10000), Nfine=2048, Ncoarse=200, T=0.5):
    x_f, x_c = make_grids(Nfine, Ncoarse)
    u0_f = random_ic_fourier(x_f, K=6, seed=seed)
    u0_c = block_average(u0_f, Ncoarse)

    # coarse truth for same final T
    t_ref, Uc_ref = reference_coarse_trajectory(u0_f, x_f, mu=mu, T=T, Ncoarse=Ncoarse, save_every=10)
    u_ref_final = Uc_ref[-1]

    out = []
    for steps in horizons:
        _, U_r = simulate_steps(u0_c, x_c, mu=mu, nsteps=steps, model=None, save_every=steps)
        _, U_n = simulate_steps(u0_c, x_c, mu=mu, nsteps=steps, model=model, save_every=steps)

        out.append({
            "steps": steps,
            "rmse_rusanov": l2_error(U_r[-1], u_ref_final),
            "rmse_nn": l2_error(U_n[-1], u_ref_final),
            "ratio": l2_error(U_n[-1], u_ref_final)/(l2_error(U_r[-1], u_ref_final)+1e-12),
            "nn_has_nan": bool(np.isnan(U_n[-1]).any()),
        })
    return out

In [18]:
stress = long_horizon_stress(model, mu=1e-2, seed=0, horizons=(2000, 5000, 20000), Ncoarse=256, T=0.5)

In [ ]:
print(stress)

{'steps': 2000, 'rmse_rusanov': 0.09085634199202502, 'rmse_nn': 0.09060361649466767, 'ratio': 0.9972184055310441, 'nn_has_nan': False}
{'steps': 5000, 'rmse_rusanov': 0.11469556407549113, 'rmse_nn': 0.11478569081709519, 'ratio': 1.0007857909879057, 'nn_has_nan': False}
{'steps': 20000, 'rmse_rusanov': 0.12306420861390018, 'rmse_nn': 0.12306764334865265, 'ratio': 1.0000279100949914, 'nn_has_nan': False}


In [11]:
def grid_refinement_study(model, mu, seed=0, Nfine=2400, Ncoarse_list=(50,100,200), T=0.2, steps=3000):
    # Nfine=2400 divisible by 50,100,200
    x_f = np.linspace(0.0, 1.0, Nfine, endpoint=False)
    u0_f = random_ic_fourier(x_f, K=6, seed=seed)

    results = []
    for Ncoarse in Ncoarse_list:
        x_c = np.linspace(0.0, 1.0, Ncoarse, endpoint=False)
        u0_c = block_average(u0_f, Ncoarse)

        # coarse truth from fine reference
        t_ref, Uc_ref = reference_coarse_trajectory(u0_f, x_f, mu=mu, T=T, Ncoarse=Ncoarse, save_every=10)
        u_ref_final = Uc_ref[-1]

        _, U_r = simulate_steps(u0_c, x_c, mu=mu, nsteps=steps, model=None, save_every=steps)
        _, U_n = simulate_steps(u0_c, x_c, mu=mu, nsteps=steps, model=model, save_every=steps)

        rmse_r = l2_error(U_r[-1], u_ref_final)
        rmse_n = l2_error(U_n[-1], u_ref_final)

        results.append({
            "Ncoarse": Ncoarse,
            "rmse_rusanov": rmse_r,
            "rmse_nn": rmse_n,
            "ratio": rmse_n/(rmse_r+1e-12),
        })
    return results

In [16]:
grid_results = grid_refinement_study(model, mu=1e-2, seed=0, Ncoarse_list=(50,100,200), T=0.1, steps=2000)

In [17]:
print(grid_results[0])
print(grid_results[1])
print(grid_results[2])

{'Ncoarse': 50, 'rmse_rusanov': 0.30310101795949496, 'rmse_nn': 0.30310102094668095, 'ratio': 1.0000000098521147}
{'Ncoarse': 100, 'rmse_rusanov': 0.3036771667557242, 'rmse_nn': 0.30370754520755133, 'ratio': 1.0001000353472458}
{'Ncoarse': 200, 'rmse_rusanov': 0.29375150670822764, 'rmse_nn': 0.2937686507519229, 'ratio': 1.0000583623991832}


In [12]:
def burgers_flux(u):
    return 0.5*u*u

def rusanov_flux(uL, uR):
    a = np.maximum(np.abs(uL), np.abs(uR))
    return 0.5*(burgers_flux(uL)+burgers_flux(uR)) - 0.5*a*(uR-uL)

def nn_flux_from_model(model, uL, uR, mu):
    uL_tf = tf.convert_to_tensor(uL.reshape(-1,1), tf.float32)
    uR_tf = tf.convert_to_tensor(uR.reshape(-1,1), tf.float32)
    log_mu = tf.fill([uL_tf.shape[0],1], tf.cast(tf.math.log(mu), tf.float32))
    out = model(tf.concat([uL_tf,uR_tf,log_mu], axis=1))
    return out.numpy().reshape(-1)

# quick probe
N = 256
rng = np.random.default_rng(0)
uL = rng.uniform(-1, 1, size=N)
uR = rng.uniform(-1, 1, size=N)
mu = 1e-2

F_r = rusanov_flux(uL, uR)
F_n = nn_flux_from_model(model, uL, uR, mu)

print("Flux stats:")
print("  max |F_nn - F_rus| =", np.max(np.abs(F_n - F_r)))
print("  mean|F_nn - F_rus| =", np.mean(np.abs(F_n - F_r)))


Flux stats:
  max |F_nn - F_rus| = 0.9311202295066499
  mean|F_nn - F_rus| = 0.21286327435511648


In [7]:
import h5py
H5_PATH = "Data/data_burgers_ref.h5"
def load_cases_from_h5(h5_path: str):
    cases = []
    with h5py.File(h5_path, "r") as f:
        cg = f["cases"]
        for name in cg.keys():
            g = cg[name]
            x = g["x"][:]
            times = g["times"][:]
            U = g["U"][:]          # [Nt, Nfine]
            mu = float(g.attrs["mu"])

            # interface samples (fine-grid adjacent pairs, stored for Phase 3)
            uL = g["uL"][:]
            uR = g["uR"][:]
            mu_vec = g["mu_vec"][:]

            cases.append(dict(name=name, x=x, times=times, U=U, mu=mu, uL=uL, uR=uR, mu_vec=mu_vec))
    return cases

cases = load_cases_from_h5(H5_PATH)
print(f"Loaded {len(cases)} cases from {H5_PATH}")
print("Example:", cases[0]["name"], "U shape:", cases[0]["U"].shape, "mu:", cases[0]["mu"])

Loaded 256 cases from Data/data_burgers_ref.h5
Example: case_00000 U shape: (43, 400) mu: 0.01878985266149949


In [22]:
# pick a case from your loaded cases exactly as in phase6
data = cases[8]  # or locate by name "case_00008"
x, times, U_ref, mu = data["x"], data["times"], data["U"], data["mu"]
dx = np.abs(x[1] - x[0])

# run using the SAME integrator used in phase6
U_nn  = integrate_to_times(U_ref[0], times, dx, mu, flux_func="nn",     flux_model=model, cfl=CFL)
U_rus = integrate_to_times(U_ref[0], times, dx, mu, flux_func="rusanov", flux_model=None,  cfl=CFL)

print("sanity:")
print("  max |U_nn - U_rus| final =", np.max(np.abs(U_nn[-1] - U_rus[-1])))
print("  L2(U_nn, U_ref) final    =", l2(U_nn[-1], U_ref[-1]))
print("  L2(U_rus, U_ref) final   =", l2(U_rus[-1], U_ref[-1]))

# interface-flux difference check at final snapshot
u = U_ref[-2]  # any snapshot
uL = u
uR = np.roll(u, -1)
F_r = rusanov_flux(uL, uR)
F_n = nn_flux(model, uL, uR, mu)
print("  max |F_nn - F_rus| =", np.max(np.abs(F_n - F_r)))

NameError: name 'CFL' is not defined

In [17]:
data.keys()

dict_keys(['name', 'x', 'times', 'U', 'mu', 'uL', 'uR', 'mu_vec'])